## The P0 Compiler
#### COMP SCI 4TB3/6TB3, McMaster University
#### Original Author: Emil Sekerinski, revised March 2021


### Revised by group 14,  March 2021

##### 1 Research Scope:
* 1. Add two new type: single-precision floating-point number(float32) and double-precision floating-point number(double64)
* 2. Type conversion between floats, doubles, and integers, including implicit type conversions for assignment, operations, relations and explicit type conversions
* 3. Add new Unicode symbols, such as √x, ⎡x⎤, ⎣x⎦, |x| for square root, floor, ceiling, absolute value
* 4. Add new operators: max and min
* 5. Add quantified expressions for arrays: sum(a[i..j]), max(a[i..j])

##### 2 Modificaitons:

* 1. Modify grammars for `factor` and `term`, add new procedure `quantifiedExpression` for the quanfified expression for arrays


```    
    factor ::= ident selector | integer | double | float | "(" expression ")" | "{" [expression {"," expression}] "}" | ("¬" | "#" | "∁" | "(int)" | "(float)" | "(double)" | "√") factor | quantifiedExpression | "⎡" factor "⎤" | "⎣" factor "⎦", | "|" factor "|"
    
    quantifiedExpression ::= ("sum" |"max") "("ident "[" integer ":" integer "]" ")"
    
    term ::= factor {("×" | "div" | "mod" | "∩" | "and" | "max" | "min") factor}

```


* 2. FIRSTFACTOR = {IDENT, INTEGER, LPAREN, NOT, CARD, COMPLEMENT, DOUBLE, FLOAT}, FIRSTEXPRESSION = {PLUS, MINUS, IDENT, INTEGER, LPAREN, NOT, CARD, COMPLEMENT, DOUBLE, FLOAT}
* 3. Modify procedure `program()` by adding float and double type
* 4. Modify procedure `factor()` to include explicit type conversions, quantified expression and unicode symbols for SQRT, FLOOR, CEILING, ABSVALUE
* 5. Modify procedure `term()` to include Float and Double type options for TIMES and DIV symbols, include max and min operators, and to allow the operands with different types for the binary operations
* 6. Modify procedure `simpleExpression()` to include Float and Double type options for PLUS and MINUS, to allow the operands with different types for the binary operations
* 7. Modify procedure `expression()` to include Float and Double type options for relation operators, to allow the operands with different types
* 8. Modify procedure `compatible()` to include conversions of type (integer, float, double)
* 9. modify procedure `statement()` to add standard procedures for write and read for float and double types 


### The P0 Grammar

    selector ::= { "[" expression "]" | "." ident}
    quantifiedExpression ::= ("sum" |"max") "("ident "[" integer ":" integer "]" ")"
    factor ::= ident selector | integer | double | float | "(" expression ")" | "{" [expression {"," expression}] "}" | ("¬" | "#" | "∁" | "(int)" | "(float)" | "(double)" | "√") factor | "⎡" factor "⎤" | "⎣" factor "⎦", | "|" factor "|" | qantifiedExpression |
    term ::= factor {("×" | "div" | "mod" | "∩" | "and" | "max" | "min") factor}
    simpleExpression ::= ["+" | "-"] term {("+" | "-" | "∪" | "or") term}
    expression ::= simpleExpression
        {("=" | "≠" | "<" | "≤" | ">" | "≥"| "∈" | "⊆" | "⊇") simpleExpression}
    statementList ::= statement {";" statement}
    statementBlock ::= statementList {statementList}
    statementSuite ::= statementList | INDENT statementBlock DEDENT
    statement ::=
        ident selector ":=" expression |
        ident {"," ident} (":=" expression {"," expression} |
        "←" ident "(" [expression {"," expression}] ")") |
        "if" expression "then" statementSuite ["else" statementSuite] |
        "while" expression "do" statementSuite
    type ::=
        ident |
         "[" expression ".." expression "]" "→" type |
          "(" typedIds ")" |
          "set" "[" expression ".." expression "]"
    typedIds ::= ident {"," ident} ":" type {"," ident {"," ident} ":" type}.
    declarations ::= 
        {"const" ident "=" expression}
        {"type" ident "=" type}
        {"var" typedIds}
        {"procedure" ident "(" [typedIds] ")" [ "→" "(" typedIds ")" ] body}
    body ::= INDENT declarations (statementBlock | INDENT statementBlock DEDENT) DEDENT
    program ::= declarations "program" ident body

In [ ]:
import nbimporter; nbimporter.options["only_defs"] = False
import SC  #  used for SC.init, SC.sym, SC.val, SC.error
from SC import TIMES, DIV, MOD, AND, PLUS, MINUS, OR, EQ, NE, LT, GT, \
    LE, GE, PERIOD, COMMA, COLON, NOT, LPAREN, RPAREN, LBRAK, RBRAK, \
    LARROW, RARROW, LBRACE, RBRACE, CARD, COMPLEMENT, UNION, INTERSECTION, \
    ELEMENT, SUBSET, SUPERSET, DOTDOT, THEN, DO, BECOMES, INTEGER, IDENT, \
    SEMICOLON, ELSE, IF, WHILE, CONST, TYPE, VAR, SET, PROCEDURE, PROGRAM, \
    INDENT, DEDENT, EOF, getSym, mark, FLOAT, DOUBLE, CONVERT_TO_INT, \
    CONVERT_TO_FLOAT, CONVERT_TO_DOUBLE, SUM, MAX, SQRT, LEFTFLOOR, LEFTCEIL, \
    RIGHTFLOOR, RIGHTCEIL, ABSVALUE, MIN

import ST  #  used for ST.init
from ST import Var, Ref, Const, Type, Proc, StdProc, Int, Bool, \
    Record, Array, Set, newDecl, find, openScope, topScope, closeScope, Float, Double

In [ ]:
def compatible(xt, yt):
    return xt == yt or (xt in {Int, Float, Double} and yt in {Int, Float, Double}) or  \
           type(xt) == Set == type(yt) or \
           type(xt) == Array == type(yt) and xt.length == yt.length and \
               compatible(xt.base, yt.base) or \
           type(xt) == Record == type(yt) and \
               all(compatible(xf.tp, yf.tp) for xf, yf in zip(xt.fields, yt.fields))

In [ ]:
# additions to FIRSTFACTOR are DOUBLE, FLOAT, SQRT, LEFTFLOOR, LEFTCEIL, ABSVALUE
# additions to FIRSTEXPRESSION are DOUBLE, FLOAT
FIRSTSELECTOR = {LBRAK, PERIOD}
FIRSTFACTOR = {IDENT, INTEGER, LPAREN, NOT, CARD, COMPLEMENT, DOUBLE, FLOAT, SQRT, LEFTFLOOR, LEFTCEIL, ABSVALUE} 
FIRSTEXPRESSION = {PLUS, MINUS, IDENT, INTEGER, LPAREN, NOT, CARD, COMPLEMENT, DOUBLE, FLOAT}
FIRSTSTATEMENT = {IDENT, IF, WHILE}
FIRSTTYPE = {IDENT, LPAREN}
FIRSTDECL = {CONST, TYPE, VAR, PROCEDURE}

In [ ]:
def selector(x):
    while SC.sym in {LBRAK, PERIOD}:
        if SC.sym == LBRAK:  #  x[y]
            getSym(); y = expression()
            if type(x.tp) == Array:
                if y.tp == Int:
                    if type(y) == Const and (y.val < x.tp.lower or y.val >= x.tp.lower + x.tp.length):
                        mark('index out of bounds')
                    else: x = CG.genIndex(x, y)
                else: mark('index not integer')
            else: mark('not an array')
            if SC.sym == RBRAK: getSym()
            else: mark("] expected")
        else:  #  x.f
            getSym()
            if SC.sym == IDENT:
                if type(x.tp) == Record:
                    for f in x.tp.fields:
                        if f.name == SC.val:
                            x = CG.genSelect(x, f); break
                    else: mark("not a field")
                    getSym()
                else: mark("not a record")
            else: mark("identifier expected")
    return x

`quantifiedExpression ::= ("sum" |"max") "("ident "[" integer ":" integer "]" ")"`

In [ ]:
# this function was defined and created to parse quantified expressions of form 
# ("sum" |"max") "("ident "[" integer ":" integer "]" ")"
def quantifiedExpression():
    op = SC.sym
    getSym()
    if SC.sym == LPAREN:
        getSym()
        if SC.sym == IDENT:
            x = find(SC.val)
            if type(x) == Var: 
                x = CG.genVar(x); getSym()
                if SC.sym == LBRAK:
                    getSym(); y = expression()
                    if SC.sym == COLON:
                        getSym(); z = expression()
                        if SC.sym == RBRAK:
                            getSym();
                            if SC.sym == RPAREN:
                                getSym();
                                if type(x.tp) == Array:
                                    if y.tp == Int and z.tp == Int:
                                        if type(y) == Const and type(z) == Const and (y.val < x.tp.lower or z.val > x.tp.lower + x.tp.length):
                                            mark('index out of bounds')
                                        else: x = CG.genAccumulation(op, x, y, z)
                                    else: mark('index not integer')
                                else: mark('not an array')
                            else: mark(') expected')
                        else: mark('] expected')      
                    else: mark(': expected')   
                else: mark('[ expected')    
            else: mark('variable identifier expected')
        else: mark('identifier expected')
    else: mark('( expected')
    return x

Procedure `factor()` parses

    factor ::= ident selector | integer | double | float | "(" expression ")" | "{" [expression {"," expression}] "}" | ("¬" | "#" | "∁" | "(int)" | "(float)" | "(double)" | "√") factor | quantifiedExpression | "⎡" factor "⎤" | "⎣" factor "⎦", | "|" factor "|"

and generates code for the factor if no error is reported. 

If the factor is `"(" expression ")"` or `"(int)" | "(float)" | "(double)" factor`, we need to distinguish the symbol after the "(". So we check if the symbol is `IDENT` and the value of the indenfier is "int" or "float" or "double", if yes, we need to parse the factor followed after the type conversion indentifier. 

In [ ]:
def factor():
    if SC.sym == IDENT:
        x = find(SC.val)
        if type(x) == Var: x = CG.genVar(x); getSym()
        elif type(x) == Const: x = Const(x.tp, x.val); x = CG.genConst(x); getSym()
        else: mark('variable or constant identifier expected')
        x = selector(x)
    # the type number is separated into INTEGER, DOUBLE, and FLOAT    
    # if it is integer
    elif SC.sym == INTEGER:
        x = Const(Int, SC.val); x = CG.genConst(x); getSym()
    # if it is double
    elif SC.sym == DOUBLE:
        x = Const(Double, SC.val); x = CG.genConst(x); getSym() 
    # if it is float
    elif SC.sym == FLOAT:
        x = Const(Float, SC.val); x = CG.genConst(x); getSym()   
    elif SC.sym == LPAREN:
        getSym();
        # LPAREN can begin an explicit conversion to type integer, float, or double, or an expression
        # if it is "(int)" | "(float)" | "(double)") factor
        if SC.sym == IDENT and (SC.val == 'int' or SC.val == 'float' or SC.val == 'double'):
            if SC.val == 'int': 
                op = CONVERT_TO_INT
            elif SC.val == 'float':
                op = CONVERT_TO_FLOAT
            elif SC.val == 'double':
                op = CONVERT_TO_DOUBLE
            getSym();
            if SC.sym == RPAREN: 
                getSym();
                x = factor()
                if x.tp not in {Int, Float, Double}: mark('not number')
                elif type(x) == Const:
                    if op == CONVERT_TO_INT: x.tp = Int; x.val = int(x.val)
                    elif op == CONVERT_TO_FLOAT: x.tp = Float; x.val = float(x.val)
                    elif op == CONVERT_TO_DOUBLE: x.tp = Double; x.val = float(x.val)
                else: x = CG.genUnaryOp(op, x) 
            else: mark(') expected')                       
        else: 
            x = expression()
            if SC.sym == RPAREN: getSym()
            else: mark(') expected')
    elif SC.sym == LBRACE:
        getSym()
        if SC.sym in FIRSTEXPRESSION:
            y = expression() 
            if y.tp == Int: x = CG.genUnaryOp(SET, y)
            else: mark('not integer')
            while SC.sym == COMMA:
                getSym(); y = expression()
                if y.tp == Int: y = CG.genUnaryOp(SET, y)
                else: mark("not integer")
                x = CG.genBinaryOp(UNION, x, y)
        else: x = Const(Set(0, 32), 0); x = CG.genConst(x)
        if SC.sym == RBRACE: getSym()
        else: mark('} expected')
    elif SC.sym == NOT:
        getSym(); x = factor()
        if x.tp != Bool: mark('not boolean')
        elif type(x) == Const: x.val = 1 - x.val # constant folding
        else: x = CG.genUnaryOp(NOT, x)
    elif SC.sym in {CARD, COMPLEMENT}:
        op = SC.sym; getSym(); x = factor()
        if type(x.tp) == Set: x = CG.genUnaryOp(op, x)
        else: mark('set expected')
    # if it is an quantified expression
    elif SC.sym in {SUM, MAX}:
        x = quantifiedExpression()
    # Factor can be preceded by the unaryOp Square root, Floor, Ceiling, or Absolute value left symbols
    # besides Square root, Factor is superceded by the right symbol for each operator
    elif SC.sym == SQRT:
        getSym(); x = factor()
        if (x.tp != Double) and (x.tp != Float): mark ('not a float or double type')
        else: x = CG.genUnaryOp(SQRT, x)
    elif SC.sym == LEFTFLOOR:
        getSym(); x = factor()
        if (x.tp != Double) and (x.tp != Float): mark ('not a float or double type')
        else: x = CG.genUnaryOp(LEFTFLOOR, x)
        if SC.sym == RIGHTFLOOR: getSym()
        else: mark('⎦ expected')
    elif SC.sym == LEFTCEIL:
        getSym(); x = factor()
        if (x.tp != Double) and (x.tp != Float): mark ('not a float or double type')
        else: x = CG.genUnaryOp(LEFTCEIL, x)
        if SC.sym == RIGHTCEIL: getSym()
        else: mark('⎤ expected')
    elif SC.sym == ABSVALUE:
        getSym(); x = factor()
        if (x.tp != Double) and (x.tp != Float): mark ('not a float or double type')
        else: x = CG.genUnaryOp(ABSVALUE, x)
        if SC.sym == ABSVALUE: getSym()
        else: mark('| expected')
    else: mark('expression expected')
    return x

  ` term ::= factor {("×" | "div" | "mod" | "∩" | "and" | "max" | "min") factor}`

In [ ]:
def term():
    x = factor()
    while SC.sym in {TIMES, DIV, MOD, INTERSECTION, AND, MAX, MIN}:
        op = SC.sym; getSym();
        if op == AND and type(x) != Const: x = CG.genUnaryOp(AND, x)
        y = factor() # x op y
        # only integer types can be operands for MOD
        if (x.tp == Int == y.tp) and op == MOD:
            if type(x) == Const == type(y): # constant folding
                x.val = x.val % y.val
            else: x = CG.genBinaryOp(op, x, y)
        # the operands for TIMES and DIV will convert implicitly to the highest precision operand
        elif ((x.tp == Int) or (x.tp == Float) or (x.tp == Double)) \
            and ((y.tp == Int) or (y.tp == Float) or (y.tp == Double)) and op in {TIMES, DIV}:
            # for Const types the result variable will convert to highest precision operand
            typ = Int
            if x.tp == Float or y.tp == Float: typ = Float
            elif x.tp == Double or y.tp == Double: typ = Double
            if type(x) == Const == type(y): # constant folding
                if op == TIMES: x.val = x.val * y.val;
                else: 
                    if x.tp == Int == y.tp: x.val = x.val // y.val;
                    else: x.val = x.val / y.val;
                x.tp = typ
            else: x = CG.genBinaryOp(op, x, y)
        elif op == INTERSECTION and type(x.tp) == Set == type(y.tp):
            x = CG.genBinaryOp(op, x, y)
        elif op == AND and x.tp == Bool == y.tp:
            if type(x) == Const: # constant folding
                if x.val: x = y # if x is true, take y, else x
            else: x = CG.genBinaryOp(AND, x, y)
        # the binary operator MAX, and MIN, return max value operations and min value operations
        elif op in (MAX, MIN) and x.tp in (Int, Float, Double) and y.tp in (Int, Float, Double):
            x = CG.genBinaryOp(op, x, y)
        else: mark('bad type')
    return x

In [ ]:
def simpleExpression():
    if SC.sym == PLUS:
        getSym(); x = term()
    elif SC.sym == MINUS:
        getSym(); x = term()
        # x must be any of the number types
        if (x.tp != Int) and (x.tp != Float) and (x.tp != Double) : mark('bad type')
        elif type(x) == Const: x.val = - x.val # constant folding
        else: x = CG.genUnaryOp(MINUS, x)
    else: x = term()
    while SC.sym in {PLUS, MINUS, UNION, OR}:
        op = SC.sym; getSym()
        if op == OR and type(x) != Const: x = CG.genUnaryOp(OR, x)
        y = term() # x op y
        # the operands for PLUS and MINUS will convert implicitly to the highest precision operand
        if ((x.tp == Int) or (x.tp == Float) or (x.tp == Double)) and ((y.tp == Int) or (y.tp == Float) or (y.tp == Double)) \
            and op in {PLUS, MINUS}:
            # for Const types the result variable will convert to highest precision operand
            typ = Int
            if x.tp == Float or y.tp == Float: typ = Float
            elif x.tp == Double or y.tp == Double: typ = Double
            if type(x) == Const == type(y): # constant folding
                if op == PLUS: x.val = x.val + y.val;
                elif op == MINUS: x.val = x.val - y.val; 
                x.tp = typ
            else: x = CG.genBinaryOp(op, x, y)
        elif x.tp == Bool == y.tp and op == OR:
            if type(x) == Const: # constant folding
                if not x.val: x = y # if x is false, take y, else x
            else: x = CG.genBinaryOp(OR, x, y)
        else: print(x, y); mark('bad type')
    return x

In [ ]:
def expression():
    x = simpleExpression()
    while SC.sym in {EQ, NE, LT, LE, GT, GE, ELEMENT, SUBSET, SUPERSET}:
        op = SC.sym; getSym(); 
        if op in (EQ, NE, LT, LE, GT, GE):
            y = simpleExpression() # x op y
            # if the types of the simpleExressions are allowed to be different, it will do type conversions when generating the code.  
            if x.tp in (Int, Bool, Float, Double) and y.tp in (Int, Bool, Float, Double):
                if type(x) == Const and type(y) == Const: # constant folding
                    if op == EQ: x.val = x.val == y.val
                    elif op == NE: x.val = x.val != y.val
                    elif op == LT: x.val = x.val < y.val
                    elif op == LE: x.val = x.val <= y.val
                    elif op == GT: x.val = x.val > y.val
                    elif op == GE: x.val = x.val >= y.val
                    x.tp = Bool
                # relations will accept any number type as operands
                elif x.tp in (Int, Float, Double) and y.tp in (Int, Float, Double): 
                    x = CG.genRelation(op, x, y)
                elif x.tp == Bool == y.tp:
                    x = CG.genRelation(op, x, y)
                else: mark('bad type')    
            else: mark('bad type')
        elif (op == ELEMENT and x.tp == Int) or \
            (op in (SUBSET, SUPERSET) and type(x.tp) == Set):
            x = CG.genUnaryOp(op, x); y = simpleExpression()
            if type(y.tp) == Set: x = CG.genRelation(op, x, y)
            else: mark('set expected')
        else: mark('bad type')
    return x

In [ ]:
def statementList():
    x = statement()
    while SC.sym == SEMICOLON:
        getSym(); y = statement(); x = CG.genSeq(x, y)
    return x

In [ ]:
def statementBlock():
    x = statementList()
    while SC.sym in FIRSTSTATEMENT:
        if not SC.newline: mark('new line expected')
        y = statementList(); x = CG.genSeq(x, y)
    return x

In [ ]:
def statementSuite():
    if SC.sym in FIRSTSTATEMENT: x = statementList()
    elif SC.sym == INDENT:
        getSym(); x = statementBlock()
        if SC.sym == DEDENT: getSym();
        else: mark('dedent or new line expected')
    else: mark("indented statement expected")
    return x

For assignment, implicit conversions take only place from smaller to larger types, which means int can be converted to float and double implicitly, float can be converted to double. However, double can only be converted to float and int explicitly.

In [ ]:
def statement():
    if SC.sym == IDENT: # x := y, y(...), x ← y(...)
        # type(x) == Proc, StdProc: check no result parameters needed; call, y := true, x
        # type(x) ≠ Proc, StdProc: x := selector():
        #   sym == BECOMES: assignment; call := false
        #   sym == LARROW: check result paramter match, type(y) is Proc, StdProc, 
        x = find(SC.val)
        if type(x) in {Proc, StdProc}: # procedure call without result
            if x.res != []: mark('variable for result expected')
            getSym(); call, xs, y = True, [], x
        elif type(x) == Var: # assignment or procedure call with result
            x = find(SC.val); x = CG.genVar(x); getSym()
            if SC.sym in FIRSTSELECTOR:
                xs = [CG.genLeftAssign(selector(x))] # array or field update
            else: # multiple assignment or procedure call with result
                xs = [CG.genLeftAssign(x)]
                while SC.sym == COMMA:
                    getSym();
                    if SC.sym == IDENT:
                        x = find(SC.val)
                        if x.name not in {x.name for x in xs}:
                            if type(x) == Var:
                                xs += [CG.genLeftAssign(CG.genVar(x))]; getSym()
                            else: mark('variable identifier expected')
                        else: mark('duplicate variable identifier')
                    else: mark('identifier expected')
            if SC.sym == BECOMES:
                # in any assignment, the right hand side will implicitly convert to the type of the left hand side
                # the position in the array ys is tracked to match with the position in array xs 
                i = 0
                getSym(); call = False; ys = [CG.genRightAssign(expression(), xs[i].tp)]; i = i + 1 # xs := ys 
                while SC.sym == COMMA: getSym(); ys += [CG.genRightAssign(expression(), xs[i].tp)]; i = i + 1
                if len(xs) == len(ys):
                    for x, y in zip(reversed(xs), reversed(ys)):
                        # x and y can be different type
                        if compatible(x.tp, y.tp): x = CG.genAssign(x, y)
                        else: mark('incompatible assignment')
                else: mark('unbalanced assignment')
            elif SC.sym == LARROW:
                getSym()
                if SC.sym == IDENT: y = find(SC.val); getSym(); call = True
                else: mark('procedure identifier expected')
                if type(y) in {Proc, StdProc}:
                    if len(xs) == len(y.res):
                        for x, r in zip(xs, y.res):
                            if not compatible(x.tp, r.tp): mark('incompatible call')
                    else: mark('unbalanced call')
                else: mark('procedure expected')
            else: mark(':= or ← expected')
        else: mark("variable or procedure expected")
        if call: # call y(ap) or xs ← y(ap)
            fp, ap, i = y.par, [], 0   #  list of formals, list of actuals
            if SC.sym == LPAREN: getSym()
            else: mark("'(' expected")
            if SC.sym in FIRSTEXPRESSION:
                a = expression()
                if i < len(fp):
                    if compatible(fp[i].tp, a.tp):
                        ap.append(CG.genActualPara(a, fp[i], i))
                    else: mark('incompatible parameter')
                else: mark('extra parameter')
                i = i + 1
                while SC.sym == COMMA:
                    getSym()
                    a = expression()
                    if i < len(fp):
                        if compatible(fp[i].tp, a.tp):
                            ap.append(CG.genActualPara(a, fp[i], i))
                        else: mark('incompatible parameter')
                    else: mark('extra parameter')
                    i = i + 1
            if SC.sym == RPAREN: getSym()
            else: mark("')' expected")
            if i < len(fp): mark('too few parameters')
            elif type(y) == StdProc: 
                if y.name == 'read': x = CG.genRead(x)   
                elif y.name == 'write': x = CG.genWrite(a)    
                elif y.name == 'writeln': x = CG.genWriteln()
            else: x = CG.genCall(xs, y, ap)
    elif SC.sym == IF:
        getSym(); x = expression();
        if x.tp == Bool: x = CG.genThen(x)
        else: mark('boolean expected')
        if SC.sym == THEN: getSym()
        else: mark("'then' expected")
        y = statementSuite()
        if SC.sym == ELSE:
            getSym()
            y = CG.genElse(x, y)
            z = statementSuite()
            x = CG.genIfElse(x, y, z)
        else:
            x = CG.genIfThen(x, y)
    elif SC.sym == WHILE:
        getSym(); t = CG.genWhile(); x = expression()
        if x.tp == Bool: x = CG.genDo(x)
        else: mark('boolean expected')
        if SC.sym == DO: getSym()
        else: mark("'do' expected")
        y = statementSuite()
        x = CG.genWhileDo(t, x, y)
    else: mark('statement expected')
    return x

In [ ]:
def typ():
    if SC.sym == IDENT:
        ident = SC.val; x = find(ident)
        if type(x) == Type: x = Type(x.val); getSym()
        else: mark('type identifier expected')
    elif SC.sym == LBRAK:
        getSym(); x = expression()
        if SC.sym == DOTDOT: getSym()
        else: mark("'..' expected")
        y = expression()
        if SC.sym == RBRAK: getSym()
        else: mark("']' expected")
        if SC.sym == RARROW: getSym()
        else: mark("'→' expected")
        z = typ().val;
        if type(x) != Const or x.val < 0: mark('bad lower bound')
        elif type(y) != Const or y.val < x.val: mark('bad upper bound')
        else: x = Type(CG.genArray(Array(z, x.val, y.val - x.val + 1)))
    elif SC.sym == LPAREN:
        getSym(); openScope(); typedIds()
        if SC.sym == RPAREN: getSym()
        else: mark("')' expected")
        r = topScope(); closeScope()
        x = Type(CG.genRec(Record(r)))
    elif SC.sym == SET:
        getSym();
        if SC.sym == LBRAK: getSym()
        else: mark ("'[' expected")
        x = expression()
        if SC.sym == DOTDOT: getSym()
        else: mark("'..' expected")
        y = expression()
        if SC.sym == RBRAK: getSym()
        else: mark("']' expected")
        if type(x) != Const: mark('bad lower bound')
        elif type(y) != Const or y.val < x.val: mark('bad upper bound')
        else: x = Type(CG.genSet(Set(x.val, y.val - x.val + 1)))
    else: mark('type expected')
    return x

In [ ]:
def typedIds():
    if SC.sym == IDENT: tid = [SC.val]; getSym()
    else: mark("identifier expected")
    while SC.sym == COMMA:
        getSym()
        if SC.sym == IDENT: tid.append(SC.val); getSym()
        else: mark('identifier expected')
    if SC.sym == COLON: getSym()
    else: mark("':' expected")
    tp = typ().val
    for i in tid: newDecl(i, Var(tp))
    while SC.sym == COMMA:
        getSym()
        if SC.sym == IDENT: tid = [SC.val]; getSym()
        else: mark("identifier expected")
        while SC.sym == COMMA:
            getSym()
            if SC.sym == IDENT: tid.append(SC.val); getSym()
            else: mark('identifier expected')
        if SC.sym == COLON: getSym()
        else: mark("':' expected")
        tp = typ().val
        for i in tid: newDecl(i, Var(tp))

In [ ]:
def declarations(allocVar):
    while SC.sym == CONST:
        getSym()
        if SC.sym == IDENT: ident = SC.val; getSym()
        else: mark("constant name expected")
        if SC.sym == EQ: getSym()
        else: mark("= expected")
        x = expression()
        if type(x) == Const: newDecl(ident, x)
        else: mark('expression not constant')
    while SC.sym == TYPE:
        getSym()
        if SC.sym == IDENT: ident = SC.val; getSym()
        else: mark("type name expected")
        if SC.sym == EQ: getSym()
        else: mark("= expected")
        x = typ(); newDecl(ident, x)  #  x is of type ST.Type
    start = len(topScope())
    while SC.sym == VAR:
        getSym(); typedIds()
    var = allocVar(topScope(), start)
    while SC.sym == PROCEDURE:
        getSym()
        if SC.sym == LPAREN:
            getSym()
            if SC.sym == IDENT: r = SC.val; getSym()
            else: mark("identifier expected")
            if SC.sym == COLON: getSym()
            else: mark("':' expected")
            tp = typ().val
            if SC.sym == RPAREN: getSym()
            else: mark(") expected")
        else: r = None
        if SC.sym == IDENT: ident = SC.val; getSym()
        else: mark("procedure name expected")
        newDecl(ident, Proc([], [])) #  entered without parameters
        sc = topScope(); openScope() # new scope for parameters and body
        if r: newDecl(r, Var(tp))
        if SC.sym == LPAREN: getSym()
        else: mark("( expected")
        if SC.sym == IDENT: typedIds()
        fp = topScope()
        if SC.sym == RPAREN: getSym()
        else: mark(") expected")
        d = len(fp)
        if SC.sym == RARROW:
            getSym()
            if SC.sym == LPAREN: getSym()
            else: mark('( expected')
            typedIds()
            if SC.sym == RPAREN: getSym()
            else: mark(') expected')
        sc[-1].par, sc[-1].res = fp[:d], fp[d:] #  procedure parameters updated
        para = CG.genProcStart(ident, fp[:d], fp[d:])
        body(ident, para); closeScope() #  scope for parameters and body closed
    return var

In [ ]:
def body(ident, para):
    if SC.sym == INDENT: getSym()
    else: mark('indent expected')
    start = len(topScope())
    local = declarations(CG.genLocalVars)
    CG.genProcEntry(ident, para, local)
    if SC.sym in FIRSTSTATEMENT: x = statementBlock()
    elif SC.sym == INDENT:
        getSym(); x = statementBlock()
        if SC.sym == DEDENT: getSym()
        else: mark('dedent or new line expected')
    else: mark('statement expected')
    CG.genProcExit(x, para, local)
    if SC.sym == DEDENT: getSym()
    else: mark('dedent or new line expected')
    return x

In [ ]:
def program():
    newDecl('boolean', Type(CG.genBool(Bool)))
    newDecl('integer', Type(CG.genInt(Int)))
    newDecl('double', Type(CG.genDouble(Double)))
    newDecl('float', Type(CG.genFloat(Float)))
    newDecl('true', Const(Bool, 1))
    newDecl('false', Const(Bool, 0))
    newDecl('read', StdProc([], [Var(Double)]))
    newDecl('write', StdProc([Var(Double)], []))
    newDecl('writeln', StdProc([], []))
    CG.genProgStart()
    declarations(CG.genGlobalVars)
    if SC.sym == PROGRAM: getSym()
    else: mark("'program' expected")
    ident = SC.val
    if SC.sym == IDENT: getSym()
    else: mark('program name expected')
    openScope(); CG.genProgEntry(ident); x = body(ident, 0)
    closeScope(); x = CG.genProgExit(x)
    return x

In [ ]:
def compileString(src, dstfn = None, target = 'wat'):
    global CG
    if target == 'wat': import CGwat as CG
    else: print('unknown target'); return
    try:
        SC.init(src); ST.init(); p = program()
        if dstfn == None: print(p)
        else:
            with open(dstfn, 'w') as f: f.write(p)
    except Exception as msg:
        # raise Exception(str(msg))
        print(msg)

In [ ]:
def compileFile(srcfn, target = 'wat'):
    if srcfn.endswith('.p'):
        with open(srcfn, 'r') as f: src = f.read()
        dstfn = srcfn[:-2] + '.s'
        compileString(src, dstfn, target)
    else: print("'.p' file extension expected")